In [1]:
import pandas as pd
data=pd.read_csv("training_cleaned_na.csv")
Y = data.pop('classLabel').values
val=pd.read_csv("val_cleaned_na.csv")
y_true= val.pop('classLabel').values

In [2]:
#replace na values with the highest occuring class or median
replace={}
for column in data.columns:
    if data[column].dtype == type(object):
        replace[column] = data[column].value_counts().index[0]
    else:
        replace[column] = data[column].median()
        
def fillna(data):
    for column in data.columns:
        if data[column].dtype == type(object):
            data[column] = data[column].fillna(replace[column])
        else:
            data[column] = data[column].fillna(replace[column])
    return
fillna(data)
fillna(val)

In [3]:
#fit a labelling function for categorical data
from sklearn import preprocessing as pre
encoders={}
for column in data.columns:
    if data[column].dtype == type(object):
        le = pre.LabelEncoder()
        data[column] = le.fit_transform(data[column])
        encoders[column]=le

In [4]:
def label(data):
    for column in data.columns:
        if data[column].dtype == type(object):
            data[column] = encoders[column].transform(data[column])
    return

label(val)

In [5]:
#train different classifiers to compare them
from sklearn import tree
X = data.values
clf = tree.DecisionTreeClassifier(max_depth=4,min_samples_leaf=15)
clf = clf.fit(X, Y)

In [6]:
tree.export_graphviz(clf) 

/home/amira/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/tree/export.py:399: DeprecationWarning: out_file can be set to None starting from 0.18. This will be the default in 0.20.
  DeprecationWarning)


In [7]:
#choose a subset of features for the classifier
#using all features will reault in a low accuracy
from sklearn.feature_selection import SelectKBest,f_classif
x_new=SelectKBest(f_classif,k=5).fit(data,Y)


In [8]:
from sklearn.neighbors import KNeighborsClassifier
neigh_clf = KNeighborsClassifier(n_neighbors=5)
X=x_new.transform(data)
neigh_clf.fit(X, Y); 

In [9]:
from sklearn import svm
svm_clf = svm.LinearSVC()
X=x_new.transform(data)
svm_clf.fit(X, Y) ;

In [10]:
from sklearn.naive_bayes import GaussianNB
nb_clf = GaussianNB()
X=x_new.transform(data)
nb_clf.fit(X, Y); 

In [11]:
from sklearn.linear_model import LogisticRegression
lr_clf=LogisticRegression()
lr_clf.fit(X, Y); 

In [12]:
#We will compare using accuracy.The data is balanced and there is no signfigance for false positives or false negatives
#Therefore accuracy is suffcient for this task
from sklearn import metrics as m
y_pred=clf.predict(val)
print(m.accuracy_score(y_true,y_pred))
print(m.accuracy_score(Y,clf.predict(data)))

0.89
0.8714285714285714


In [13]:
y_pred=neigh_clf.predict(x_new.transform(val))
print(m.accuracy_score(y_true,y_pred))
print(m.accuracy_score(Y,neigh_clf.predict(x_new.transform(data))))

0.815
0.8836734693877552


In [14]:
y_pred=svm_clf.predict(x_new.transform(val))
print(m.accuracy_score(y_true,y_pred))
print(m.accuracy_score(Y,svm_clf.predict(X)))

0.85
0.8571428571428571


In [15]:
y_pred=nb_clf.predict(x_new.transform(val))
print(m.accuracy_score(y_true,y_pred))
print(m.accuracy_score(Y,nb_clf.predict(X)))

0.87
0.8510204081632653


In [16]:
y_pred=lr_clf.predict(x_new.transform(val))
print(m.accuracy_score(y_true,y_pred))
print(m.accuracy_score(Y,lr_clf.predict(X)))

0.86
0.8551020408163266


In [17]:
#The best model is the decision tree classifier
def predict(input_data):
    input_data=input_data.drop(columns="variable18")
    input_data=input_data.drop(columns="variable17")
    input_data=input_data.drop(columns="variable19")
    input_data=input_data.drop(columns="variable4")
    input_data=input_data.drop(columns="variable11")
    fillna(input_data)
    label(input_data)
    return clf.predict(input_data)